In [7]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
from PIL import Image, ImageOps
from keras.preprocessing.image import load_img
from keras.models import load_model

#model1 = load_model('Customised_Model.hd5')
model1 = load_model('CNN_Model.hd5')
#model1 = load_model("gender_detection.model")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

gender_dict = {0:'Male', 1:'Female'}

def face_extractor(frame):
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = frame[y:y+h, x:x+w]
        im = Image.fromarray(cropped_face)
        im.save(r"Images/1.jpg")
    return frame

def dataframe_creat():
    BASE_DIR = r'Images/'
    image_paths = []
    for filename in (os.listdir(BASE_DIR)):
        image_path = os.path.join(BASE_DIR, filename)
        image_paths.append(image_path)
    df = pd.DataFrame()
    df['image'] = image_paths
    df.head()
    return df

def extract_features(images):
    features = []
    for image in (images):
        img = load_img(image,color_mode = "grayscale")
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 128, 128, 1)
    return features

def final_funct():
    df = dataframe_creat()
    X = extract_features(df['image'])
    X = X/255.0
    img_index = 0
    pred = model1.predict(X[img_index].reshape(1,128, 128, 1))
    pred_gender = gender_dict[round(pred[0][0][0])]
    pred_age = round(pred[1][0][0])
    return pred_gender,pred_age    

In [24]:
cap=cv2.VideoCapture(0)

while True:
    _,frame=cap.read()
    frames = face_extractor(frame)
    pred_gender,pred_age = final_funct()
    final_txt = pred_gender+' '+str(pred_age)
    cv2.putText(frame,str(final_txt), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Face Cropper',frames)

    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break
cap.release()
cv2.destroyAllWindows()

file = glob.glob(r'Images\*.jpg')
for filename in file:
    os.unlink(filename)